In [1]:
# importacion de librerias
import pandas as pd
import numpy as np

# visualizaciones
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/raw/bank-additional.csv', index_col=0)

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,date,latitude,longitude,id_
0,NaN,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,1,...,1.1,"93,994","-36,4","4,857",5191,no,2-agosto-2019,41.495,-71.233,089b39d8-e4d0-461b-87d4-814d71e0e079
1,57.0,services,MARRIED,high.school,NaN,0.0,0.0,telephone,149,1,...,1.1,"93,994","-36,4",NaN,5191,no,14-septiembre-2016,34.601,-83.923,e9d37224-cb6f-4942-98d7-46672963d097
2,37.0,services,MARRIED,high.school,0.0,1.0,0.0,telephone,226,1,...,1.1,"93,994","-36,4","4,857",5191,no,15-febrero-2019,34.939,-94.847,3f9f49b5-e410-4948-bf6e-f9244f04918b
3,40.0,admin.,MARRIED,basic.6y,0.0,0.0,0.0,telephone,151,1,...,1.1,"93,994","-36,4",NaN,5191,no,29-noviembre-2015,49.041,-70.308,9991fafb-4447-451a-8be2-b0df6098d13e
4,56.0,services,MARRIED,high.school,0.0,0.0,1.0,telephone,307,1,...,1.1,"93,994","-36,4",NaN,5191,no,29-enero-2017,38.033,-104.463,eca60b76-70b6-4077-80ba-bc52e8ebb0eb


## SELECCION DE COLUMNAS 
vamos a eliminar las columnas que no son relevantes para nuestro analisis segun lo que vimos hasta ahora.
- default: practicamente solo tenemos datos de clientes con inclumpimiento de pago y nulos. no es de utilidad para el estudio por la falta de datos, por el momento los eliminamos.
- pdays: no parece relevante para el estudio, la eliminamos. 
- previous: No es muy informativa, ya que el 75% de los clientes no fueron contactados antes.
- latitude y longitude: no tiene sentido, la eliminamos
- contact: parece un dato irrelevante para el estudio, la eliminamos
- poutcome: no nos da mucha infirmacion, tenemos 90% de datos sin informacion, la eliminamos

In [4]:
columnas_eliminar = ['default','pdays','previous','latitude','longitude','contact','poutcome']

In [5]:
df = df.drop(columns=columnas_eliminar)

In [6]:
df.columns

Index(['age', 'job', 'marital', 'education', 'housing', 'loan', 'duration',
       'campaign', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y', 'date', 'id_'],
      dtype='object')

## LIMPIEZA DE DATOS

## COLUMNA AGE
como tenemos un 12% de nulos, que es moderado, no demasiado, vamos a rellenar los valores faltantes con una imputacion aleatoria apartir de los valores existentes

In [7]:
n_nulos = df['age'].isnull().sum()

valores_aleatorios = np.random.choice(df['age'].dropna(), size=n_nulos, replace=True)
df.loc[df['age'].isnull(), 'age'] = valores_aleatorios

## COLUMNA MARTITAL 
tenemos solo un 0,2% de valores nulos, los eliminamos directamente.

In [8]:
df = df.dropna(subset=['marital'])

## COLUMNA EDUCATION
vamos a agrupar en rangos mas amplios para reducir el ruido y mejorar la interpretacion.
- Higher: university.degree, professional.course --> educación superior o técnica
- Secondary: high.school --> educacion secundaria completa
- Basic: basic.4y, basic.6y, basic.9y --> educación básica
- Illiterate: illiterate
- Unknown: NaN	

In [9]:
education_map = {
    'university.degree': 'Higher',
    'professional.course': 'Higher',
    'high.school': 'Secondary',
    'basic.4y': 'Basic',
    'basic.6y': 'Basic',
    'basic.9y': 'Basic',
    'illiterate': 'Illiterate'
}

df['education_grouped'] = df['education'].map(education_map)
df['education_grouped'] = df['education_grouped'].fillna('Unknown')

## COLUMNA HOUSING
tenemos solo 2,39% de valores nulos, los eliminamos

In [10]:
df = df.dropna(subset=['housing'])

## COLUMNA LOAN
teniamos nulos que se eliminaron porque coincidian con los de la columna housing, lo dejamos asi

## COLUMNA DURATION
eliminamos los valores de duradion 0 que no tienen sentido y para los outliers extremos vamos a utilizar un percentil alto como limite (0.99)

In [11]:
df = df[df['duration'] > 0]

In [12]:
df = df[df['duration'] < df['duration'].quantile(0.99)]

## COLUMNA CAMPAIGN
limitamos a 10 el valor total para eliminar a los outliers, en una nueva columna por el momento

In [13]:
df['campaign_capped'] = df['campaign'].apply(lambda x: x if x <= 10 else 10)

## COLUMNA EMP.VAR.RATE
vamos a crear una variable categorica para agrupar por etiquetas el contexto economico

In [14]:
df['emp_var_context'] = pd.cut(df['emp.var.rate'],
                               bins=[-10, -2, 0, 2, 10],
                               labels=['Recesión fuerte', 'Recesión leve', 'Estable', 'Crecimiento'])

## COLUMNAS 'cons.price.idx', 'cons.conf.idx','euribor3m', 'nr.employed'
tenemmos que reemplazar las , por . para convertir a columnas numericas

In [15]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.replace(',','.')
    try:
        df[col] = df[col].astype(float)
    except:
        pass

In [16]:
df.describe(include='number').T.round(2)

,count,mean,std,min,25%,50%,75%,max
age,41466.0,39.98,10.45,17.00,32.00,38.00,47.00,98.00
housing,41466.0,0.54,0.50,0.00,0.00,1.00,1.00,1.00
loan,41466.0,0.16,0.36,0.00,0.00,0.00,0.00,1.00
duration,41466.0,243.62,212.15,1.00,102.00,178.00,313.00,1267.00
campaign,41466.0,2.57,2.77,1.00,1.00,2.00,3.00,43.00
emp.var.rate,41466.0,0.08,1.57,-3.40,-1.80,1.10,1.40,1.40
cons.price.idx,41009.0,93.57,0.58,92.20,93.08,93.44,93.99,94.77
cons.conf.idx,41466.0,-40.51,4.64,-50.80,-42.70,-41.80,-36.40,-26.90
euribor3m,32551.0,3.61,1.74,0.63,1.34,4.86,4.96,5.04
nr.employed,41466.0,5166.84,72.38,4963.60,5099.10,5191.00,5228.10,5228.10


## COLUMNA cons.price.idx	
rellenamos con la mediana los valores nulos, son muy pocos

In [17]:
df['cons.price.idx'] = df['cons.price.idx'].fillna(df['cons.price.idx'].median())

## COLUMNA cons.conf.idx
agrupamos en rangos el indice de confianza del consumidor para manejarlo mejor como una columna categorica

In [18]:
df['conf_idx_cat'] = pd.qcut(df['cons.conf.idx'], q=4, labels=['Very Low', 'Low', 'Medium', 'High'])

## COLUMNA EURIBOR3M
convertimos en una categorica y agrupamos por etiquetas, como tenemos muchos nulos, agrupamos en una variable de unknown

In [19]:
df['euribor_cat'] = pd.cut(df['euribor3m'], bins=[0, 2, 4, 6], labels=['Low', 'Medium', 'High'])
df['euribor_cat'] = df['euribor_cat'].cat.add_categories('Unknown')
df['euribor_cat'] = df['euribor_cat'].fillna('Unknown')

## COLUMNA DATE
pasar a formato fecha

In [20]:
meses = {'enero':'01',
         'febrero':'02',
         'marzo':'03',
         'abril':'04',
         'mayo':'05',
         'junio':'06',
         'julio':'07',
         'agosto':'08',
         'septiembre':'09',
         'octubre':'10',
         'noviembre':'11',
         'diciembre':'12'}

In [21]:
df.replace({'date':meses}, regex=True, inplace= True)

In [22]:
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41466 entries, 0 to 27570
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   age                41466 non-null  float64       
 1   job                41139 non-null  object        
 2   marital            41466 non-null  object        
 3   education          39724 non-null  object        
 4   housing            41466 non-null  float64       
 5   loan               41466 non-null  float64       
 6   duration           41466 non-null  int64         
 7   campaign           41466 non-null  int64         
 8   emp.var.rate       41466 non-null  float64       
 9   cons.price.idx     41466 non-null  float64       
 10  cons.conf.idx      41466 non-null  float64       
 11  euribor3m          32551 non-null  float64       
 12  nr.employed        41466 non-null  float64       
 13  y                  41466 non-null  object        
 14  date       

## guardamos los datos limpios

In [23]:
df.to_csv('../DATA/OUTPUT/bank-additional_limpio.csv', index=False)